In [163]:
# disable compiler warnings
import os

# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from typing import List
import datetime
from tqdm.notebook import tqdm


from tensorflow.python.client import device_lib
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # FATAL
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))


Num GPUs Available:  []


In [164]:
(train_ds, val_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

#tfds.show_examples(train_ds, ds_info)

In [165]:
class BasicConv(tf.keras.Model):
    def __init__(self, optimizer=tf.keras.optimizers.Adam()):
        super(BasicConv, self).__init__()

        self.optimizer = optimizer
        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.metrics_list = [[tf.keras.metrics.CategoricalAccuracy(name="test_accuracy"),
                             tf.keras.metrics.Mean(name="test_loss"),
                             tf.keras.metrics.Mean(name="test_frob_norm")],
                             [tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
                             tf.keras.metrics.Mean(name="train_loss"),
                             tf.keras.metrics.Mean(name="train_frob_norm")]]
  
        self.pooling = tf.keras.layers.MaxPooling2D()
        self.my_layers = [
                        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
                        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D()),
                        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
                        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAvgPool2D()),
                        tf.keras.layers.Dense(10, activation='softmax')
                        ]



    @tf.function
    def call(self, x, training=False):
        for layer in self.my_layers:
            tf.print(x)
            x = layer(x)
        return x

    @tf.function
    def compute_frobenius(self):
        frobenius_norm = tf.zeros((1,))
        for var in self.trainable_variables:
            frobenius_norm += tf.norm(var, ord="euclidean")
        return frobenius_norm

    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list

    # 4. reset all metrics objects
    def reset_metrics(self, num):
        for metric in self.metrics[num]:
            metric.reset_states()


    # train_step method
    @tf.function
    def train_step(self, data):
        img, label = data
        
        # compute output and loss, train the variables
        with tf.GradientTape() as tape:
            output = self(img, training=True)
            loss = self.loss_function(label, output)
            
        # update trainable variables
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update metrics
        self.metrics_list[1][0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[1][1].update_state(loss)
        self.metrics_list[1][2].update_state(self.compute_frobenius())
        
        # return a dict with metric information
        return {m.name : m.result() for m in self.metrics_list[1]}



    # test_step method
    @tf.function
    def test_step(self, data):
        img, label = data

        # compute output and loss, without training
        output = self(img, training=False)
        loss = self.loss_function(label, output)

        # update metrics
        self.metrics_list[0][0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[0][1].update_state(loss)
        self.metrics_list[0][2].update_state(self.compute_frobenius())

        # return a dict with metric information 
        return {m.name : m.result() for m in self.metrics_list[0]}

In [166]:

def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer

In [167]:
# gets in a dataset and returns target values
def prepare_data(dataset, batch_size):

    # get list of all targets
    targets = []
    for images, labels in dataset:  # only take first element of dataset
        targets.append(labels.numpy())

    # convert into tensor
    targets_tensor = tf.convert_to_tensor(targets, dtype=tf.int32)

    # range_vals is only used to create the change between positive and negative signs for the summation
    range_vals = tf.range(len(targets))

    # target_digits is a placeholder
    target_digits = tf.random.uniform((10,), minval=0, maxval=10, dtype=tf.dtypes.int32)

    # this creates the alternating positve and negative signes by checkign whether the entry index modulo 2 is zero (i.e. even entries are positive, uneven ones negative)
    # check the tf.where documentation if the usage is confusing for you!
    alternating_target_numbers = tf.where(tf.math.floormod(range_vals,2)==0, targets_tensor, -targets_tensor)

    # finally we can compute the cumulative sums!
    c_sum = tf.math.cumsum(alternating_target_numbers)

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img/128.)-1., target))
    
    # replace targets
    for (img, label), sum in zip(dataset, c_sum):
        label = tf.one_hot(sum, depth=10)

    dataset = dataset.cache()
    dataset = dataset.shuffle(4096)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [168]:
# trains the model by iterating through the dataset and applying training_step method epochs time
def training_loop(model, train_ds, test_ds, epochs, train_summary_writer, memory):
    metrics = []

    # iterate over epochs
    for epoch in tqdm(range(epochs)):

        # train steps on all batches in the training data
        for (img, label) in train_ds:
            metrics = model.train_step((img, label))
            
            # keep data in summary with metrics
            with train_summary_writer.as_default():
                for metric in model.metrics_list[1]:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        for (key, value) in metrics.items():
            memory[key].append(value.numpy())

        memory = test_loop(model=model,
                           test_ds=test_ds,
                           val_summary_writer=val_summary_writer,
                           memory=memory)
        
        # print current metric values and reset the metrics
        tf.print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])
        model.reset_metrics(1)

    return memory


# tests overall performance of model
def test_loop(model, test_ds, val_summary_writer, memory):
    metrics = []
    # test steps on every item in test dataset
    for (img, label) in tqdm(test_ds):
        metrics = model.test_step((img, label))
        
        # keep data with metrics
        with val_summary_writer.as_default():
            for metric in model.metrics_list[0]:
                tf.summary.scalar(f"{metric.name}", metric.result(), step=1)

    for (key, value) in metrics.items():
        memory[key].append(value.numpy())

    print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])

    model.reset_metrics(0)

    return memory

# visualize accuracy, loss and frobenius norm
def visualization(accuracies, losses, frobenius, name):
    plt.figure()
    line1, = plt.plot(accuracies, "b")
    line2, = plt.plot(losses, "r")

    frob_new = frobenius/np.max(frobenius) * np.max(losses)
    line3, = plt.plot(frob_new, "y" )

    plt.xlabel("Training steps")
    plt.ylabel("Loss/Accuracy")
    plt.legend((line1, line2, line3),("Accuracy", "Loss", "Frobenius Norm"))
    plt.savefig(name)
    plt.show()


In [169]:
# overall hyperparameters to compare with and without overfitting precautions methods
epochs = 15
batch_size = 32

## no augmentation, normal model

In [170]:
train_summary_writer, val_summary_writer = create_summary_writers(config_name="UNTOUCHED")

model = BasicConv()


for img, label in train_ds.take(1):
    print(img.shape)
    print(label.shape)

train_dataset = prepare_data(train_ds, batch_size)
val_dataset = prepare_data(val_ds, batch_size)

for img, label in train_dataset.take(1):
    print(img.shape)
    print(label.shape)

(28, 28, 1)
()
(32, 28, 28, 1)
(32,)


In [171]:
tfds.benchmark(train_dataset, batch_size=batch_size)

print("\n************ Training UNTOUCHED ************\n")

memory = {"test_accuracy" : [],
          "test_loss" : [],
          "test_frob_norm" : [],
          "train_accuracy" : [],
          "train_loss" : [],
          "train_frob_norm" : []
          }

memory = training_loop(model,
                       train_ds=train_dataset,
                       test_ds=val_dataset,
                       epochs=tf.constant(epochs),
                       train_summary_writer=train_summary_writer,
                       memory=memory)

visualization(memory["train_accuracy"], memory["train_loss"], memory["train_frob_norm"], "aug_train")


************ Summary ************



  0%|          | 0/1875 [00:00<?, ?it/s]

Examples/sec (First included) 84975.83 ex/sec (total: 60032 ex, 0.71 sec)
Examples/sec (First only) 516.42 ex/sec (total: 32 ex, 0.06 sec)
Examples/sec (First excluded) 93096.26 ex/sec (total: 60000 ex, 0.64 sec)

************ Training UNTOUCHED ************



  0%|          | 0/15 [00:00<?, ?it/s]

ValueError: in user code:

    File "C:\Users\JULIAN~1\AppData\Local\Temp/ipykernel_18304/291518793.py", line 60, in train_step  *
        loss = self.loss_function(label, output)
    File "c:\Users\Julian Unland\anaconda3\envs\ki_uebungen\lib\site-packages\keras\losses.py", line 139, in __call__  **
        losses = call_fn(y_true, y_pred)
    File "c:\Users\Julian Unland\anaconda3\envs\ki_uebungen\lib\site-packages\keras\losses.py", line 243, in call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    File "c:\Users\Julian Unland\anaconda3\envs\ki_uebungen\lib\site-packages\keras\losses.py", line 1787, in categorical_crossentropy
        return backend.categorical_crossentropy(
    File "c:\Users\Julian Unland\anaconda3\envs\ki_uebungen\lib\site-packages\keras\backend.py", line 5119, in categorical_crossentropy
        target.shape.assert_is_compatible_with(output.shape)

    ValueError: Shapes (32,) and (32, 10) are incompatible
